<a href="https://colab.research.google.com/github/leechearien/object-detection/blob/main/car16_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 모델 import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D ,Flatten
from tensorflow.keras.utils import to_categorical 
import numpy as  np
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from PIL import Image
import os, glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# 분류 대상 카테고리 선택하기
caltech_dir = "/content/gdrive/MyDrive/car5"

categories = categories = ["AvanteAD","Carnival","G4rexton",
                           "GenesisG80","GrandeurIG","K5JF",
                           "Kona","LFSonata","Morning",
                           "Porter2","QM6","Ray","SM6",
                           "SantafeTM","Spark","Tivoli"]

nb_classes = len(categories) # nb_classes에 분류될 클래스를 개수만큼 변수에 저장해둔다.
print(nb_classes)

# 이미지 크기 지정
image_w = 64
image_h = 64
pixels = image_w * image_h * 3 # pixel은 rgb값이 모두 들어가니 3을 곱해준다.

# 이미지 데이터 읽어 들이기
X = [] # 실제 이미지 데이터
Y = [] # 레이블 데이터

for idx, cat in enumerate(categories):
    # 레이블 생성
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

      # 이미지
    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.JPG")    # 확장자가 jpg인 파일을 찾기
    for i, f in enumerate(files):
        img = Image.open(f)                  # 이미지 파일을 읽어서
        img = img.convert("RGB")             # RGB 형식으로 변환하고
        img = img.resize((image_w, image_h)) # 지정해줬던 크기로 resize한다.
        data = np.asarray(img)               # numpy 배열 데이터로 변환한다.
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)

# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = \
    train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)

# 데이터를 저장합니다.
np.save("/content/gdrive/MyDrive/car_16/16obj.npy", xy)
print("ok,", len(Y))

In [ ]:
# 카테고리 지정하기
categories = categories = ["AvanteAD","Carnival","G4rexton",
                           "GenesisG80","GrandeurIG","K5JF",
                           "Kona","LFSonata","Morning",
                           "Porter2","QM6","Ray","SM6",
                           "SantafeTM","Spark","Tivoli"]
nb_classes = len(categories) # nb_classes에 분류될 클래스를 개수만큼 변수에 저장해둔다.

# 이미지 크기 지정하기
image_w = 128
image_h = 128

# 데이터 불러오기
X_train, X_test, y_train, y_test = np.load('/content/gdrive/MyDrive/car_16/16obj.npy', allow_pickle=True)

# 데이터 정규화하기
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255
print('X_train shape:', X_train.shape)
print('X_text shape:',X_test.shape)


# 모델 구성하기
model = Sequential()
model.add(Convolution2D(32, 3, 3, padding='same',input_shape=X_train.shape[1:])) # 1층
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3,padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# 모델을 설정하기
model.compile(loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['acc'])

# 콜백을 정의하기
callbacks = [EarlyStopping(monitor = 'val_acc', patience = 10, verbose = 1)]

# 모델 훈련하기
history= model.fit(X_train, y_train,
         batch_size = 100,
         validation_data = (X_test, y_test),
         epochs = 50)
         #callbacks = callbacks) 
  
# 모델 평가하기
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

# 모델을 저장하기
model.save('/content/drive/MyDrive/car_16/car_CNN.h5')

In [ ]:
# 모델 시각화

model = load_model('/content/drive/MyDrive/car_16/car_CNN.h5')

model.evaluate(X_test, y_test)

train_acc_list=history.history['acc']
train_acc_list
test_acc_list=history.history['val_acc']
test_acc_list

x = np.arange( len(train_acc_list) )
plt.plot( x, train_acc_list, label='train acc')
plt.plot( x, test_acc_list, label='test acc',  linestyle='--')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.show()